<a href="https://colab.research.google.com/github/CodeeSam/Bioinformatics/blob/main/Metagonome_pilot_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing necessary packages
!pip install biopython
!pip install transformers
!pip install tensorflow==2.18.0

# Importing required libraries
import os
import numpy as np
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

print("Environment setup complete!")
print("TensorFlow version:", tf.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 26.9 MB/s eta 0:00:00
Environment setup complete!
TensorFlow version: 2.18.0


In [ ]:
# Updating package lists and installing EMBOSS (which provides getorf)
!apt-get update
!apt-get install -y emboss

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,788 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,685 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1

In [ ]:
os.getcwd()

'/content'

In [ ]:
os.chdir('/content/sample_data')

In [ ]:
!ls -l

total 55548
-rwxr-xr-x 1 root root     1697 Jan  1  2000 anscombe.json
-rw-r--r-- 1 root root   301141 Mar 28 13:39 california_housing_test.csv
-rw-r--r-- 1 root root  1706430 Mar 28 13:39 california_housing_train.csv
-rw-r--r-- 1 root root 18289443 Mar 28 13:39 mnist_test.csv
-rw-r--r-- 1 root root 36523880 Mar 28 13:39 mnist_train_small.csv
-rw-r--r-- 1 root root      958 Apr  1 06:11 pilot_AMPs_test.fa
-rw-r--r-- 1 root root     2814 Apr  1 06:11 pilot_Non-AMPs_test.fa
-rw-r--r-- 1 root root    36321 Apr  1 06:11 pilot_pilot_study_seq.fasta
-rwxr-xr-x 1 root root      962 Jan  1  2000 README.md


In [ ]:
# Running getorf on the test metagenome FASTA file to extract ORFs between 15 and 150 bp
!getorf -sequence pilot_pilot_study_seq.fasta -find 0 -table 11 -minsize 15 -maxsize 150 -outseq pilot_pilot_study_seq.orfs.fa

Find and extract open reading frames (ORFs)


In [ ]:
def load_fasta(filepath):
    """Load a FASTA file into a dictionary with record IDs as keys and sequences as values."""
    return {record.id: str(record.seq) for record in SeqIO.parse(filepath, "fasta")}

# Loading candidate ORFs and known AMPs
candidate_orfs = load_fasta("/content/sample_data/pilot_pilot_study_seq.orfs.fa")
known_amp = load_fasta("/content/sample_data/pilot_AMPs_test.fa")

# Filtering out candidates that exactly match any known AMP
filtered_candidates = {k: v for k, v in candidate_orfs.items() if v not in known_amp.values()}

print("Number of candidate ORFs before filtering:", len(candidate_orfs))
print("Number of candidate ORFs after filtering known AMPs:", len(filtered_candidates))


In [ ]:
# Defining a function to read a FASTA file and tokenize sequences
def tokenize_sequences(fasta_file, max_length=512):
    #Initializing the tokenizer from a pre-trained model- ProtBert:
    tokenizer = AutoTokenizer.from_pretrained("Rostlab/prot_bert_bfd", do_lower_case=False)
    sequences = []
    ids = []
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequences.append(" ".join(list(str(record.seq))))
        ids.append(record.id)
    tokens = tokenizer(sequences, return_tensors="tf", padding='max_length', truncation=True, max_length=max_length)
    return tokens, ids

# Tokenizing the known AMP (positive) and non-AMP (negative) sequences
tokens_pos, ids_pos = tokenize_sequences("/content/sample_data/pilot_AMPs_test.fa", max_length=64)
tokens_neg, ids_neg = tokenize_sequences("/content/sample_data/pilot_Non-AMPs_test.fa", max_length=64)

print("Tokenization complete for AMP and non-AMP sequences.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Tokenization complete for AMP and non-AMP sequences.


In [ ]:
seq_object = SeqIO.parse('pilot_pilot_study_seq.fasta', 'fasta')
sequencess = []

for seq in seq_object:
    sequencess.append(seq)

sequencess
len(sequencess)

60

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained("Rostlab/prot_bert_bfd", num_labels=2)

# Preparing TensorFlow dataset for training
def prepare_dataset(tokens_pos, tokens_neg, batch_size=9):
    labels = tf.concat([
        tf.ones((tokens_pos['input_ids'].shape[0],), dtype=tf.int32),
        tf.zeros((tokens_neg['input_ids'].shape[0],), dtype=tf.int32)
    ], axis=0)
    input_ids = tf.concat([tokens_pos['input_ids'], tokens_neg['input_ids']], axis=0)
    attention_mask = tf.concat([tokens_pos['attention_mask'], tokens_neg['attention_mask']], axis=0)
    dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': input_ids, 'attention_mask': attention_mask}, labels))
    return dataset.shuffle(10).batch(batch_size)

train_dataset = prepare_dataset(tokens_pos, tokens_neg, batch_size=9)

tf_model.h5:   0%|          | 0.00/1.85G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at Rostlab/prot_bert_bfd and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
labels = tf.concat([
        tf.ones((tokens_pos['input_ids'].shape[0],), dtype=tf.int32),
        tf.zeros((tokens_neg['input_ids'].shape[0],), dtype=tf.int32)
    ], axis=0)

In [ ]:
labels

<tf.Tensor: shape=(109,), dtype=int32, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)>

In [ ]:
# Compile and fine-tune the model
#optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ##binary_crossentropy ***
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])


model.summary()

# Fine-tuning for a small number of epochs (for the pilot)
history = model.fit(train_dataset, epochs=5)

In [ ]:
# Tokenizing candidate ORFs from filtered_candidates
def tokenize_candidate_sequences(candidate_dict, max_length=64):
    sequences = []
    ids = []
    for seq_id, seq in candidate_dict.items():
        sequences.append(" ".join(list(seq)))
        ids.append(seq_id)
    tokenizer = AutoTokenizer.from_pretrained("Rostlab/prot_bert_bfd", do_lower_case=False)
    tokens = tokenizer(sequences, return_tensors="tf", padding='max_length', truncation=True, max_length=max_length)
    return tokens, ids

tokens_candidates, candidate_ids = tokenize_candidate_sequences(filtered_candidates, max_length=64)

# Predicting using the fine-tuned model
predictions = model.predict(tokens_candidates)
# Get probabilities from logits using softmax
probabilities = tf.nn.softmax(predictions.logits, axis=-1).numpy()[:, 1]  # probability for AMP class

# Creatin a DataFrame for results
results = pd.DataFrame({
    "Candidate_ID": candidate_ids,
    "Sequence": list(filtered_candidates.values()),
    "Prediction_Probability": probabilities
})

# Set threshold (0.5) for classification
results["Predicted_Label"] = (results["Prediction_Probability"] >= 0.5).astype(int)
print("Candidate AMP Predictions:")
print(results)

38/38 [==============================] - 1132s 30s/step
Candidate AMP Predictions:
              Candidate_ID                                      Sequence  \
0      JBEVUU010000001.1_1                               RALMVKVRLPTSWVC   
1      JBEVUU010000001.1_2                              KYGYRPHGCARQRGWD   
2      JBEVUU010000001.1_3                              NFDWWQYRCSSARSAL   
3      JBEVUU010000001.1_4                                SLCSGSGYLNFVCI   
4      JBEVUU010000001.1_5  ATRLGLKLRLVAISVFQRSICSLKILIFMFWFGIFKFCLYINLL   
...                    ...                                           ...   
1202  JBEVUU010000060.1_32                                     ICSVHCFMP   
1203  JBEVUU010000060.1_33                                    FVYTPHLECC   
1204  JBEVUU010000060.1_34                                   VIMERSFAINF   
1205  JBEVUU010000060.1_35                  WNGVSQSTFSRKMQTVWHYTKFYSQLSP   
1206  JBEVUU010000060.1_36                           QKDANSLALHKVLQSVITQ   

    

In [ ]:
results.to_csv("/content/sample_data/candidate_amp_predictions.csv", index=False)
print("Candidate AMP predictions saved to 'Sample_data/candidate_amp_predictions.csv'.")

Candidate AMP predictions saved to 'Sample_data/candidate_amp_predictions.csv'.
